In [259]:
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
import os
import imageio
from PIL import Image
import glob

% matplotlib inline

In [2]:
#https://www.kaggle.com/skainkaryam/basic-data-visualization-using-pytorch-dataset
class TGSSaltDataset(data.Dataset):
    
    def __init__(self, root_path, file_list):
        self.root_path = root_path
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        if index not in range(0, len(self.file_list)):
            return self.__getitem__(np.random.randint(0, self.__len__()))
        
        file_id = self.file_list[index]
        
        image_folder = os.path.join(self.root_path, "images")
        image_path = os.path.join(image_folder, file_id + ".png")
        
        mask_folder = os.path.join(self.root_path, "masks")
        mask_path = os.path.join(mask_folder, file_id + ".png")
        
        image = np.array(imageio.imread(image_path), dtype=np.uint8)
        mask = np.array(imageio.imread(mask_path), dtype=np.uint8)
        
        return image, mask

In [3]:
df_depth = pd.read_csv('./data/train.csv')

In [4]:
train_path = "./data/train/"
train_file_list = list(df_depth['id'].values)

In [5]:
len(train_file_list)

4000

In [42]:
train_data = TGSSaltDataset(train_path, train_file_list)

In [61]:
train_path

'./data/train/'

In [ ]:
image_path

In [65]:
img = np.array(imageio.imread('./data/train/images/000e218f21.png'), dtype=np.uint8)

In [71]:
img[0][0]

array([131, 131, 131], dtype=uint8)

In [75]:
mean_img[0][1]

120.991

In [96]:
def load_img_to_df(img_path, normalize=False, mean_img=None):
    images = []
    for filename in glob.glob(f'{img_path}/*.png'): #assuming gif
        img_id = filename.split('\\')[-1].split('.')[0]
        img = np.array(imageio.imread(filename), dtype=np.float)
        if normalize:
            img -= mean_img[:,:,None]
        images.append([img_id, img])
    return pd.DataFrame(images, columns=['img_id', 'img']).set_index('img_id')

In [97]:
def load_img_to_np(img_path):
    images = []
    for filename in glob.glob(f'{img_path}/*.png'): #assuming gif
        img_id = filename.split('\\')[-1].split('.')[0]
        images.append(np.array(imageio.imread(filename), dtype=np.uint8))
    return np.r_[images]

In [98]:
np_train = load_img_to_np('./data/train/images')

In [99]:
mean_img = np_train[:,:,:,0].reshape(np_train.shape[0], -1).mean(0).reshape(101,101)

In [100]:
df_train = load_img_to_df('./data/train/images', normalize=True, mean_img=mean_img)

In [101]:
df_train['mask'] = load_img('./data/train/masks/')['img']

In [ ]:
df_test = load_img_to_df('./data/test/images', normalize=True, mean_img=mean_img)

In [108]:
np_test = np.r_[df_test.img.tolist()]

In [578]:
np_test[:,:,:,1].mean()

-0.6330799240815169

In [579]:
np_test.shape

(18000, 101, 101, 3)

In [115]:
np_train = np.r_[df_train.img.tolist()]

In [508]:
X_train, X_val, y_train, y_val = (
    train_test_split(df_train, df_train['mask'], 
                     test_size=0.20, 
                     random_state=42)
)

In [510]:
X_train = np.r_[X_train.img.tolist()]

In [511]:
X_val = np.r_[X_val.img.tolist()]

In [512]:
y_train = np.r_[y_train.tolist()]

In [513]:
y_val = np.r_[y_val.tolist()]

In [502]:
def format_train_data(X):
    X = np.moveaxis(X, -1,1)
    X = np.pad(X, [(0,0),(0,0),(0, 1), (0, 1)], mode='constant')
    return torch.from_numpy(X).float()

In [419]:
def get_train_batch(X_train, y_train, batch_size):
    batch_ids = np.random.choice(len(X_train), batch_size, replace=False)
    X_train_batch = X_train[batch_ids]
    X_train_batch = np.moveaxis(X_train_batch, -1,1)
    X_train_batch = np.pad(X_train_batch, [(0,0),(0,0),(0, 1), (0, 1)], mode='constant')
    y_train_batch = np.clip(y_train[batch_ids],0,1)
    
    return (torch.from_numpy(X_train_batch).float(), torch.from_numpy(y_train_batch).float())

In [435]:
class SaltNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(3,64,3, padding=10),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,128,3),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,256,3),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(256, 128, 2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 64, 2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, 1, 2, stride=2, padding=1),
            nn.Sigmoid()
        )
        
    def forward(self, X):
        out = self.seq(X)
        return out[:,:,:-1,:-1].squeeze()

In [487]:
saltnet = SaltNet()

In [488]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(saltnet.parameters(), lr=0.001)

In [489]:
import time
import math

In [490]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [491]:
start = time.time()

In [492]:
for i in range(2500):
    X_batch, y_batch = get_train_batch(X_train, y_train, batch_size=64)
    y_pred = saltnet(X_batch)
    optimizer.zero_grad()
    loss = loss_fn(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(f'Loss is {loss} at iter {i}, Time: {timeSince(start)}')

Loss is 0.7075795531272888 at iter 0, Time: 0m 7s
Loss is 0.6087774634361267 at iter 10, Time: 1m 32s
Loss is 0.5375742316246033 at iter 20, Time: 2m 39s
Loss is 0.5098478198051453 at iter 30, Time: 3m 46s
Loss is 0.4907909035682678 at iter 40, Time: 4m 55s
Loss is 0.5025970935821533 at iter 50, Time: 6m 5s
Loss is 0.5129656791687012 at iter 60, Time: 7m 17s
Loss is 0.3500281870365143 at iter 70, Time: 8m 35s
Loss is 0.46371912956237793 at iter 80, Time: 9m 55s
Loss is 0.38141101598739624 at iter 90, Time: 11m 10s
Loss is 0.45511162281036377 at iter 100, Time: 12m 23s
Loss is 0.4406242072582245 at iter 110, Time: 13m 34s
Loss is 0.27835792303085327 at iter 120, Time: 14m 45s
Loss is 0.2957882881164551 at iter 130, Time: 15m 56s
Loss is 0.39439597725868225 at iter 140, Time: 18m 58s
Loss is 0.3745402991771698 at iter 150, Time: 22m 18s
Loss is 0.28691571950912476 at iter 160, Time: 25m 34s
Loss is 0.29820430278778076 at iter 170, Time: 28m 50s
Loss is 0.3296818435192108 at iter 180, Tim

KeyboardInterrupt: 

In [495]:
state = {'epoch': i + 1, 'state_dict': saltnet.state_dict(),
             'optimizer': optimizer.state_dict() }
torch.save(state, 'saltnet_v1_20180808_0837.cpt')

In [580]:
X_test.shape

(800, 101, 101, 3)

In [581]:
np_test.shape

(18000, 101, 101, 3)

In [586]:
del(df_depth)

In [588]:
np_test.shape

(18000, 101, 101, 3)

In [591]:
np.split(np.arange(10), 2)

[array([0, 1, 2, 3, 4]), array([5, 6, 7, 8, 9])]

In [589]:
X_test_torch = format_train_data(np_test[:8000]).shape

MemoryError: 

In [555]:
y_test_torch = torch.from_numpy(np.clip(y_test,0,1)).float()

In [592]:
y_test_pred = []

In [598]:
len(y_test_pred)

1800

In [594]:
np_test.shape

(18000, 101, 101, 3)

In [597]:
for x in np.split(np_test,1800):
    with torch.no_grad():
        y_test_pred.append(saltnet(format_train_data(x)))

In [599]:
y_test_pred = torch.cat(y_test_pred)

In [600]:
y_test_pred = y_test_pred.ge(0.5)

In [602]:
y_test_pred.shape

torch.Size([18000, 101, 101])

In [603]:
def rlencode(x, dropna=False):
    """
    Run length encoding.
    Based on http://stackoverflow.com/a/32681075, which is based on the rle 
    function from R.
    
    Parameters
    ----------
    x : 1D array_like
        Input array to encode
    dropna: bool, optional
        Drop all runs of NaNs.
    
    Returns
    -------
    start positions, run lengths, run values
    
    """
    where = np.flatnonzero
    x = np.asarray(x)
    n = len(x)
    if n == 0:
        return (np.array([], dtype=int), 
                np.array([], dtype=int), 
                np.array([], dtype=x.dtype))

    starts = np.r_[0, where(~np.isclose(x[1:], x[:-1], equal_nan=True)) + 1]
    lengths = np.diff(np.r_[starts, n])
    values = x[starts]
    
    if dropna:
        mask = ~np.isnan(values)
        starts, lengths, values = starts[mask], lengths[mask], values[mask]
    
    return starts[::2]+1, lengths[::2]

In [640]:
y_test_pred_rle = [rlencode(x.T.flatten()) for x in y_test_pred.detach().numpy()]

(array([    1,  8065,  8166,  8465,  8565, 10027, 10029, 10130],
       dtype=int64),
 array([8062,   99,  297,   99, 1458,    1,   95,   72], dtype=int64))

In [644]:
y_test_pred_rle = [np.c_[e] for e in y_test_pred_rle]

In [652]:
y_test_pred_rle[0]

array([[    1,  8062],
       [ 8065,    99],
       [ 8166,   297],
       [ 8465,    99],
       [ 8565,  1458],
       [10027,     1],
       [10029,    95],
       [10130,    72]], dtype=int64)

In [669]:
y_test_pred_rle2 = [[i, e[0]+' '+e[1]] for i, f in enumerate(y_test_pred_rle) for e in f.astype(str)]

In [670]:
df_y_test_pred_rle2 = pd.DataFrame(y_test_pred_rle2)

In [679]:
df_y_test_pred_rle2['id'] = df_y_test_pred_rle2[0].map({i:v for i,v in enumerate(df_test.index.tolist())})

In [681]:
df_y_test_pred_rle2['rle_mask'] = df_y_test_pred_rle2[1]

In [682]:
df_y_test_pred_rle2.head()

,0,1,id,rle_mask
0,0,1 8062,0005bb9630,1 8062
1,0,8065 99,0005bb9630,8065 99
2,0,8166 297,0005bb9630,8166 297
3,0,8465 99,0005bb9630,8465 99
4,0,8565 1458,0005bb9630,8565 1458


In [618]:
y_test_pred_rle = [' '.join(np.c_[e].flatten().astype(str).tolist()) for e in y_test_pred_rle]

In [617]:
' '.join(np.c_[y_test_pred_rle[0]].flatten().astype(str).tolist())

'1 8062 8065 99 8166 297 8465 99 8565 1458 10027 1 10029 95 10130 72'

In [624]:
len(y_test_pred_rle)

18000

In [ ]:
df_test_pred = pd.DataFrame

In [626]:
def load_img_to_df(img_path, normalize=False, mean_img=None):
    images = []
    for filename in glob.glob(f'{img_path}/*.png'): #assuming gif
        img_id = filename.split('\\')[-1].split('.')[0]
        img = ''
        if normalize:
            img -= mean_img[:,:,None]
        images.append([img_id, img])
    return pd.DataFrame(images, columns=['img_id', 'img']).set_index('img_id')

In [627]:
df_test = load_img_to_df('./data/test/images')

In [629]:
df_test['rle_mask'] = y_test_pred_rle

In [636]:
df_test.reset_index()[['img_id', 'rle_mask']].to_csv('submission_20180808_01.csv', index=False)

In [637]:
del(np_test)

In [639]:
y_test_pred[0]

tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]], dtype=torch.uint8)

In [685]:
df_y_test_pred_rle2[['id', 'rle_mask']].to_csv('submission_20180808_02.csv', index=False)

In [686]:
df_y_test_pred_rle2.id.nunique()

18000